In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
from matplotlib import font_manager
for font in font_manager.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)

NanumMyeongjo /usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf
NanumBarunGothic /usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf
NanumSquareRound /usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf
NanumSquare /usr/share/fonts/truetype/nanum/NanumSquareR.ttf
NanumGothic /usr/share/fonts/truetype/nanum/NanumGothicBold.ttf
NanumBarunGothic /usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf
NanumMyeongjo /usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf
NanumGothic /usr/share/fonts/truetype/nanum/NanumGothic.ttf
NanumSquare /usr/share/fonts/truetype/nanum/NanumSquareB.ttf
NanumSquareRound /usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf


In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['axes.unicode_minus'] = False

# 이미지 cropping

In [ ]:
import glob
import pandas as pd
import cv2
import json
import unicodedata

In [ ]:
order = 0
num = 0
# gb = glob.glob('/content/drive/MyDrive/공유폴더/091.차량 외관 영상 데이터/01.데이터/2.Validation/라벨링데이터/*/*/*/*.json')
df = pd.DataFrame(columns = ['size', 'brand', 'model', 'year','angle','file_path','origin_path'])

for js in gb:
    
    리어램프 = 0
    헤드램프 = 0
    차량전체 = False

    with open(js, 'r') as f:
        
        json_data = json.load(f)
        data_dict = json.dumps(json_data)
        data_dict = json.loads(data_dict)
    for obj in data_dict['learningDataInfo']['objects']:
        
        if obj['classId'] == 'P11.리어램프':
            리어램프 += 1

        if obj['classId'] == 'P10.헤드램프':
            헤드램프 += 1

        if obj['classId'] == 'P00.차량전체':
            차량전체 = True
            
            origin_img = js.replace('라벨링데이터','원천데이터').replace('json', 'jpg')
            img = cv2.imread(origin_img)
            L = int(obj['left'])
            T = int(obj['top'])
            W = int(obj['width'])
            H = int(obj['height'])
            cropped_img = img[T: T+H, L:L+W]
            model_name = origin_img[origin_img.find('원천데이터')+15:origin_img.rfind('/')] + str(num)
            model_name = unicodedata.normalize('NFKC', model_name)
            model_name = model_name.replace('/','_')
            df.loc[num,'origin_path'] = origin_img
            info = data_dict['rawDataInfo']
            df.loc[num,'size'] = info['LargeCategoryId']
            df.loc[num,'brand'] = info['MediumCategoryId']
            df.loc[num,'model'] = info['SmallCategoryId']
            df.loc[num,'year'] = info['yearId']
        
        if (리어램프 == 2) and 차량전체:
            file_path = './후면/' + model_name + '.jpg'
            df.loc[num,'file_path'] = file_path
            cv2.imwrite(file_path, cropped_img)
            df.loc[num,'angle'] = '후면'
            num +=1
            break
        
        elif (헤드램프 == 2) and 차량전체:
            file_path = './정면/' + model_name + '.jpg'
            df.loc[num,'file_path'] = file_path
            cv2.imwrite(file_path, cropped_img)
            df.loc[num,'angle'] = '정면'
            num +=1
            break
    
    if 차량전체:
        if df['angle'].isnull().any():
            file_path = './측면/' + model_name + '.jpg'
            df.loc[num,'file_path'] = file_path
            cv2.imwrite(file_path, cropped_img)
            df.loc[num,'angle'] = '측면'
            num +=1

    if order%50 ==0:        
        print(order)
    order+=1

In [ ]:
df.to_csv('angle_cropped_df.csv')